In [1]:
from datetime import date
import pandas as pd
import nselib
from nselib import capital_market
from sqlalchemy import create_engine
import numpy as np
import os
from growwapi import GrowwAPI
import pyotp
import duckdb
from pandas import json_normalize
import time

In [2]:
motherduck_token = os.getenv("MOTHERDUCK_TOKEN")
con = duckdb.connect(f"md:?motherduck_token={motherduck_token}")

In [112]:
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
CHAT_ID = '5798902540'

In [3]:
groww_api_key = os.getenv('GROWW_API_KEY')
groww_api_secret = os.getenv('GROWW_API_SECRET')
totp_gen = pyotp.TOTP(groww_api_secret)
totp = totp_gen.now()

access_token = GrowwAPI.get_access_token(groww_api_key, totp)
groww = GrowwAPI(access_token)

Ready to Groww!


In [4]:
instruments_df = groww.get_all_instruments()

instruments_priority = instruments_df.copy()
instruments_priority['priority'] = instruments_priority['exchange'].apply(lambda x: 1 if x=='NSE' else 2)
instruments_unique = instruments_priority.sort_values(['trading_symbol','priority']).drop_duplicates('trading_symbol', keep='first')

In [11]:
df = con.sql("SELECT symbol as 'Ticker', date1 as 'Date', open_price as 'Open', high_price as 'High',low_price as 'Low',close_price as 'Close', deliv_qty as 'Volume' FROM daily_nse_price where series = 'EQ' and date1 > '2025-01-01'").df()

df["Volume"] = df["Volume"].astype(float)

## Supertrend

In [ ]:
def weekly_supertrend_daily(df, atr_period=10, multiplier=3):
    """
    df: DataFrame with ['Ticker','Date','Open','High','Low','Close']
    Returns df with 'SuperTrend' and 'Trend' columns applied to daily rows, based on weekly ATR
    """
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Ticker','Date'])

    result_list = []

    for ticker, group in df.groupby('Ticker'):
        # Resample weekly to get High, Low, Close for SuperTrend calculation
        weekly = group.resample('W-FRI', on='Date').agg({
            'High':'max',
            'Low':'min',
            'Close':'last'
        }).sort_index()

        # Calculate weekly ATR
        weekly['H-L'] = weekly['High'] - weekly['Low']
        weekly['H-Cp'] = abs(weekly['High'] - weekly['Close'].shift())
        weekly['L-Cp'] = abs(weekly['Low'] - weekly['Close'].shift())
        weekly['TR'] = weekly[['H-L','H-Cp','L-Cp']].max(axis=1)
        weekly['ATR'] = weekly['TR'].rolling(atr_period, min_periods=1).mean()

        # Basic bands
        weekly['Basic_Up'] = (weekly['High'] + weekly['Low'])/2 + multiplier*weekly['ATR']
        weekly['Basic_Down'] = (weekly['High'] + weekly['Low'])/2 - multiplier*weekly['ATR']

        # Weekly SuperTrend
        weekly['SuperTrend'] = 0.0
        for i in range(1,len(weekly)):
            prev = weekly.iloc[i-1]
            if prev['SuperTrend'] < prev['Close']:
                curr_st = max(weekly.iloc[i]['Basic_Down'], prev['SuperTrend'])
            else:
                curr_st = min(weekly.iloc[i]['Basic_Up'], prev['SuperTrend'])
            weekly.at[weekly.index[i], 'SuperTrend'] = curr_st

        # ---- 2. Map weekly SuperTrend to daily data ----
        group = group.set_index('Date')
        group['SuperTrend'] = weekly['SuperTrend'].reindex(group.index, method='ffill')

        # ---- 3. Daily Trend based on daily Close vs weekly SuperTrend ----
        group['Trend'] = group['Close'] > group['SuperTrend']

        group['Ticker'] = ticker
        result_list.append(group.reset_index())

    return pd.concat(result_list, ignore_index=True)

In [ ]:
daily_with_weekly_st = weekly_supertrend_daily(df, atr_period=10, multiplier=2)

In [ ]:
latest_supertrend = daily_with_weekly_st.groupby("Ticker").tail(1)

In [78]:
con.register("latest_supertrend", latest_supertrend).execute("create or replace table latest_supertrend as select * from latest_supertrend")

## 📊 Buy list
---

In [5]:
con.sql("SELECT date1,count(*) from daily_nse_price group by date1 order by date1 desc").df()

,date1,count_star()
0,2025-09-25,2980
1,2025-09-24,2990
2,2025-09-23,2982
3,2025-09-22,3039
4,2025-09-19,2979
...,...,...
1164,2021-01-07,2017
1165,2021-01-06,2040
1166,2021-01-05,2049
1167,2021-01-04,2055


In [6]:
allstocks= pd.read_excel("data/allstocks.xlsx")

In [5]:
NSE_ISIN = pd.read_csv("data/NSE_ISIN.csv")
NSE_ISIN.columns = NSE_ISIN.columns.str.strip()

In [8]:
latest_supertrend = con.sql("select * from latest_supertrend").df()

In [12]:
print(latest_supertrend)

           Date      Ticker      Open      High       Low     Close    Volume  \
0    2025-09-25   20MICRONS    227.20    228.00    223.65    225.34   57577.0   
1    2025-09-25  21STCENMGM     53.01     53.01     51.36     52.05    2134.0   
2    2025-09-25      360ONE   1025.00   1029.20   1012.00   1022.10  552856.0   
3    2025-09-25   3IINFOLTD     24.54     24.54     23.40     23.85  398508.0   
4    2025-09-25     3MINDIA  29520.00  29545.00  28790.00  28920.00    3698.0   
...         ...         ...       ...       ...       ...       ...       ...   
2412 2025-09-25        ZOTA   1474.40   1499.00   1450.00   1484.60   12376.0   
2413 2025-09-25       ZUARI    302.25    302.25    291.00    302.25  339794.0   
2414 2025-09-25    ZUARIIND    395.85    416.00    387.40    403.90  944922.0   
2415 2025-09-25   ZYDUSLIFE   1035.45   1042.40   1016.45   1019.25  734930.0   
2416 2025-09-25   ZYDUSWELL    486.90    487.20    470.60    474.20   68704.0   

        SuperTrend  Trend  

In [13]:
price_threshold = 0.03
volume_threshold = 2.0

latest_date = df["Date"].max()

In [14]:
results = []

for ticker, group in df.groupby("Ticker"):
    group = group.sort_values("Date").reset_index(drop=True)

    # Calculate daily % price change (Close vs Previous Close)
    group["Pct_Change"] = group["Close"].pct_change()

    # Calculate rolling average volume (last 20 days)
    group["Avg_Volume"] = group["Volume"].rolling(window=20, min_periods=5).mean()

    # Signal: price up > threshold & volume > threshold * avg_volume
    group["Signal"] = (group["Pct_Change"] > price_threshold) & (group["Volume"] > volume_threshold * group["Avg_Volume"])

    results.append(group)

result_df = pd.concat(results)

signals = result_df[result_df["Signal"]]
latest_signals = signals[signals["Date"] == latest_date]

In [15]:
latest_signals = latest_signals.merge(
    instruments_unique[["trading_symbol","isin"]],
    left_on="Ticker",
    right_on="trading_symbol",
    how="inner"
)

In [16]:
latest_signals = pd.merge(
    latest_signals,
    allstocks,
    left_on="isin",         # column name in latest_signals
    right_on="ISIN",     # column name in allstocks
    how="left"           # inner join (only matching tickers)
)

In [17]:
latest_signals = latest_signals.merge(
    latest_supertrend[["Ticker","SuperTrend","Trend"]],
    on="Ticker",         # column name in latest_signals
    how="left"           # inner join (only matching tickers)
)

In [18]:
latest_signals.shape

(35, 58)

In [19]:
latest_signals.to_csv("latest_signals.csv", index=False)

In [43]:
filtered = latest_signals[
    (latest_signals["Market Capitalization"] > 1000) &
    #(latest_signals["ROCE Annual 3Yr Avg %"] > 15) &
    #(latest_signals['Long Term Debt To Equity Annual'] < 0.5) &
    #(latest_signals['Promoter holding latest %'] > 60) &
    (latest_signals['Trend'] == True) &
    (latest_signals['Promoter holding pledge percentage % Qtr'] < 0.01) &
    (latest_signals['Net Profit Qtr Growth YoY %'] > 0) &
    (latest_signals['Operating Revenue growth TTM %'] > 10) &
    (latest_signals['Cash EPS 5Yr Growth %'] > 15) &
    (latest_signals['EPS TTM Growth %'] > 15) &
    (latest_signals["PEG TTM PE to Growth"] > 0) &
    (latest_signals["PEG TTM PE to Growth"] < 2)
]

In [44]:
filtered.shape

(1, 58)

In [41]:
filtered.to_csv("filtered.csv", index=False)

In [113]:


def send_dataframe_via_telegram(df, bot_token, chat_id, caption="DataFrame"):
    import matplotlib.pyplot as plt
    import requests
    
    if df.empty:
        telegram = requests.post(
            f"https://api.telegram.org/bot{bot_token}/sendMessage",
            data={"chat_id": chat_id, "text": f"Nothing in {caption} list today."}
        )
        return telegram.json()
    # Render DataFrame as image
    fig, ax = plt.subplots(figsize=(6, 0.5 + 0.3*len(df)))  # auto height
    ax.axis("off")
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc="center",
        loc="center"
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    filename = "table.png"
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    
    # Send to Telegram
    url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"
    with open(filename, "rb") as f:
        res = requests.post(url, data={"chat_id": chat_id, "caption": caption}, files={"photo": f})
    return res.json()

In [22]:

send_dataframe_via_telegram(filtered[["Stock Name"]], BOT_TOKEN, CHAT_ID, "Buy")

{'ok': True,
 'result': {'message_id': 42,
  'from': {'id': 7252728340,
   'is_bot': True,
   'first_name': 'DailyNseDataLoad',
   'username': 'DpsStockAnalysis_bot'},
  'chat': {'id': 5798902540,
   'first_name': 'Daksh',
   'last_name': 'Singh',
   'type': 'private'},
  'date': 1758689264,
  'text': 'Nothing in Buy list today.'}}

## Sell List
---

In [80]:

# Step 1: Create holdings DataFrame (from your response)
holdings_response = groww.get_holdings_for_user(timeout=5)


In [92]:
holdings = pd.DataFrame(holdings_response["holdings"])
holdings = holdings[["isin", "trading_symbol", "quantity", "average_price"]]

instruments_priority = instruments_df.copy()
instruments_priority['priority'] = instruments_priority['exchange'].apply(lambda x: 1 if x=='NSE' else 2)
instruments_unique = instruments_priority.sort_values(['trading_symbol','priority']).drop_duplicates('trading_symbol', keep='first')

holdings = holdings.merge(
    instruments_unique[['isin', 'exchange']],
    on='isin',
    how='left'
)


In [93]:
# Columns from get_quote you want to add
quote_cols = [
    "bid_quantity", "bid_price", "day_change", "day_change_perc",
    "upper_circuit_limit", "lower_circuit_limit", "last_price", "high_trade_range",
    "low_trade_range", "volume", "week_52_high", "week_52_low", "market_cap"
]

# Initialize empty dict to store quote data
quotes_data = {col: [] for col in quote_cols}

# Loop through holdings
for idx, row in holdings.iterrows():
    try:
        quote_response = groww.get_quote(
            exchange=row["exchange"],  # NSE or BSE
            segment=groww.SEGMENT_CASH,
            trading_symbol=row["trading_symbol"]
        )
        
        # Add data to quotes_data dict
        for col in quote_cols:
            # Handle nested OHLC if needed
            if col in quote_response:
                quotes_data[col].append(quote_response[col])
            else:
                quotes_data[col].append(None)
        
        # Optional: small delay to avoid API limits
        time.sleep(0.1)

    except Exception as e:
        print(f"Error fetching {row['trading_symbol']}: {e}")
        for col in quote_cols:
            quotes_data[col].append(None)

# Convert quotes_data to DataFrame
quotes_df = pd.DataFrame(quotes_data)

# Concatenate with holdings
holdings = pd.concat([holdings.reset_index(drop=True), quotes_df], axis=1)



In [95]:
holdings["ltp"] = holdings["last_price"]  # use last_price as LTP
holdings["current_value"] = holdings["quantity"] * holdings["ltp"]
holdings["pnl_percent"] = (holdings["ltp"] - holdings["average_price"]) / holdings["average_price"] * 100
holdings["day_gain"] = holdings["quantity"] * holdings["day_change"]

In [97]:
holdings = pd.merge(
    holdings,
    allstocks,
    left_on="isin",         # column name in latest_signals
    right_on="ISIN",     # column name in allstocks
    how="left"           # inner join (only matching tickers)
)

In [100]:
holdings = pd.merge(
    holdings,
    latest_supertrend[["Ticker","SuperTrend","Trend"]],
    left_on="trading_symbol",         # column name in latest_signals
    right_on="Ticker",         
    how="left"           # inner join (only matching tickers)
)

In [118]:
# Restrict "Stock Name" to first 2 words
holdings["Stock Name"] = holdings["Stock Name"].str.split().str[:2].str.join(" ")
# Further restrict to first word if length > 20
holdings["Stock Name"] = holdings["Stock Name"].apply(lambda x: x.split()[0] if len(x) > 15 else x)


In [119]:
# Select specific columns and filter rows where Trend is False
sell_list = holdings.loc[
    holdings["Trend"] == False,
    ["Stock Name", "current_value", "day_change_perc", "pnl_percent"]
]

# Format numeric columns
sell_list["current_value"] = sell_list["current_value"].astype(int)
sell_list["day_change_perc"] = sell_list["day_change_perc"].round(2)
sell_list["pnl_percent"] = sell_list["pnl_percent"].round(2)

In [121]:
print(holdings)

            isin trading_symbol  quantity  average_price exchange  \
0   INE024F01011      SHILCTECH     283.0        5263.65      BSE   
1   INE0PPK01015     ORIENTTECH    1330.0         374.72      NSE   
2   INE0R4701017     ALPEXSOLAR    1200.0        1176.22      NSE   
3   INE162B01018       LUMAXIND     464.0        3239.38      NSE   
4   INE205B01031         ELECON    2503.0         600.06      NSE   
5   INE230A01023        EIHOTEL    3688.0         405.54      NSE   
6   INE281B01032       LLOYDSME    1478.0        1013.19      NSE   
7   INE397D01024     BHARTIARTL     765.0        1965.86      NSE   
8   INE410P01011             NH     852.0        1755.35      NSE   
9   INE417T01026      POLICYBZR     788.0        1901.97      NSE   
10  INE419D01026          CLSEL    1507.0         331.31      NSE   
11  INE451A01017       FORCEMOT     112.0       13287.76      NSE   
12  INE495S01016            HPL    2635.0         568.22      NSE   
13  INE646L01027         INDIGO   

In [ ]:
send_dataframe_via_telegram(sell_list, BOT_TOKEN, CHAT_ID, "Sell")

DEBUG:matplotlib:matplotlib data path: /home/vscode/.local/lib/python3.11/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/vscode/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/vscode/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/vscode/.cache/matplotlib/fontlist-v390.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='/home/vscode/.local/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono-Oblique.ttf', name='DejaVu Sans Mono', style='oblique', variant='normal', weight=400, stretch='normal', size=

{'ok': True,
 'result': {'message_id': 47,
  'from': {'id': 7252728340,
   'is_bot': True,
   'first_name': 'DailyNseDataLoad',
   'username': 'DpsStockAnalysis_bot'},
  'chat': {'id': 5798902540,
   'first_name': 'Daksh',
   'last_name': 'Singh',
   'type': 'private'},
  'date': 1758862641,
  'photo': [{'file_id': 'AgACAgUAAxkDAAMvaNYdMWdhO7xjM5DxblSHi4J-SRMAAh3DMRt5XbBWoW_zJUGc96QBAAMCAANzAAM2BA',
    'file_unique_id': 'AQADHcMxG3ldsFZ4',
    'file_size': 787,
    'width': 90,
    'height': 23},
   {'file_id': 'AgACAgUAAxkDAAMvaNYdMWdhO7xjM5DxblSHi4J-SRMAAh3DMRt5XbBWoW_zJUGc96QBAAMCAANtAAM2BA',
    'file_unique_id': 'AQADHcMxG3ldsFZy',
    'file_size': 8576,
    'width': 320,
    'height': 83},
   {'file_id': 'AgACAgUAAxkDAAMvaNYdMWdhO7xjM5DxblSHi4J-SRMAAh3DMRt5XbBWoW_zJUGc96QBAAMCAAN4AAM2BA',
    'file_unique_id': 'AQADHcMxG3ldsFZ9',
    'file_size': 18831,
    'width': 577,
    'height': 150}],
  'caption': 'Buy'}}

In [122]:
holdings.to_csv("my_data.csv", index=False)

In [124]:
latest_supertrend.loc[latest_supertrend["Ticker"] == "ALPEXSOLAR"]

,Date,Ticker,Open,High,Low,Close,Volume,SuperTrend,Trend


In [127]:
df.loc[df["Ticker"] == "ALPEXSOLAR"]

,Ticker,Date,Open,High,Low,Close,Volume
